## HCBB gryphons_goal_3

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import time

%matplotlib inline

## 1. What procedures could be reasonably associated with each individual Provider Type? (There are 88 types of Providers.)
- Filters for Physician/Other Supplier --Do we need a "credentials of the provider" column?
    - Entity Type = I
    - Place of Service = F and/or O
- What is the normal ‘basket’ of procedures for each Provider Type?

## 2. Which procedure codes drive the most revenue for each Professional Provider Types?
- Exclude office visits

## Stretch: Create a process that predicts if an individual professional should be Provider Type ‘Orthopedic Surgery’ based on the procedure codes they have data for.
- How often does this data-driven expected Provider Type of ‘Orthopedic Surgery’ differ from the listed Provider Type for given professional?

## Import pkl data, from first project notebook

In [ ]:
providers_and_procedures = pd.read_pickle("../data/hcpcs_providers_clean.pkl")

## begin EDA of dataset

In [ ]:
providers_and_procedures.info()

In [ ]:
providers_and_procedures.head()

In [ ]:
providers_and_procedures['Place of Service'].value_counts()

In [ ]:
providers_and_procedures['Provider Type'].unique()

In [ ]:
providers_and_procedures['HCPCS Description'].value_counts()

## Begin work on Question 1. 
- create pivot table for provider types and HCPCS descriptions

In [ ]:
providers_pivot_table = pd.pivot_table(providers_and_procedures, index = ['Provider Type', 'HCPCS Code', 
                                                                          'HCPCS Description', 'Place of Service'])

In [ ]:
providers_pivot_table.head(75)

In [ ]:
providers_pivot_table.shape

## Attempt subset of one Provider Type - 'Addiction Medicine'

In [ ]:
addict_med_type = providers_and_procedures.loc[providers_and_procedures['Provider Type'] == 'Addiction Medicine']

In [ ]:
addict_med_type.shape

In [ ]:
addict_med_type.head()

In [ ]:
addiction_pivot_table = pd.pivot_table(addict_med_type, index = ['Provider Type', 'HCPCS Description'])

In [ ]:
addiction_pivot_table.shape

## Turn full pivot table into a dataframe

In [ ]:
## df = table.reset_index() 
prov_desc_df = providers_pivot_table.reset_index()
prov_desc_df.head(5)

In [ ]:
prov_desc_df['HCPCS Description'].value_counts()

## Delete HCPCS codes for office visits

In [ ]:
prov_desc_df_no_office = prov_desc_df.loc[~prov_desc_df['HCPCS Code'].str.contains('992..|993..', regex=True)]

In [246]:
prov_desc_df_no_office.shape

(51680, 12)

In [ ]:
prov_desc_df_no_office

## Calculate percentage of the services provided for each type.

In [247]:
## sort the provider types with the number of services ranked 
prov_desc_df_sorted = prov_desc_df_no_office.sort_values(by = ['Provider Type', 'Number of Services'], ascending = (True, False))

In [248]:
prov_desc_df_sorted

,Provider Type,HCPCS Code,HCPCS Description,Place of Service,Average Medicare Allowed Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,Average Submitted Charge Amount,National Provider Identifier,Number of Distinct Medicare Beneficiary/Per Day Services,Number of Medicare Beneficiaries,Number of Services
71,Addiction Medicine,90847,"Family psychotherapy including patient, 50 min...",O,102.760000,75.680820,80.779673,120.013687,1.700878e+09,1622.0,128.0,1622.0
83,Addiction Medicine,95004,"Injection of allergenic extracts into skin, ac...",O,6.360746,4.800754,5.395866,8.425672,1.295891e+09,39.0,39.0,1340.0
33,Addiction Medicine,82575,Creatinine clearance measurement to test for k...,O,12.970000,12.710000,12.710000,25.000000,1.942314e+09,957.0,41.0,957.0
156,Addiction Medicine,G0397,Alcohol and/or substance (other than tobacco) ...,O,69.620000,53.877956,55.292240,139.240000,1.124073e+09,817.0,283.0,817.0
80,Addiction Medicine,93798,Physician services for outpatient heart rehabi...,F,13.580000,10.207731,10.823529,100.000000,1.740270e+09,595.0,31.0,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...
55755,Vascular Surgery,71101,X-ray of ribs on one side of body including th...,F,13.000000,9.263636,9.774545,47.545455,1.578745e+09,11.0,11.0,11.0
55791,Vascular Surgery,73522,"X-ray of both hips with pelvis, 3-4 views",F,14.340000,11.240000,11.810000,44.000000,1.578745e+09,11.0,11.0,11.0
55861,Vascular Surgery,75956,Radiological supervision and interpretation of...,F,331.350000,259.780000,278.840000,2666.000000,1.326207e+09,11.0,11.0,11.0
55956,Vascular Surgery,92950,Attempt to restart heart and lungs,F,198.150000,152.240000,150.530000,845.000000,1.538141e+09,11.0,11.0,11.0


In [249]:
## makes a list of datframes for each single provider type with its hcpcs codes from the original df
provider_type_df_list = list(prov_desc_df_sorted.groupby(['Provider Type'])) 

In [250]:
len(provider_type_df_list) ## check for 88 provider types--one is lost from dropping office visits

87

In [ ]:
provider_type_df_list[0]

In [257]:
## iterate over list
provider_threshold_list = []

for group, provider in provider_type_df_list:
    total_hcpcs = len(provider) ## number of hcpcs for each provider
    keep_rows = round(total_hcpcs * .80) ## can change this number to keep desired percentage
    keep_provider = provider.head(keep_rows)
    provider_threshold_list.append(keep_provider)

providers_final = pd.concat(provider_threshold_list)

In [258]:
len(provider_threshold_list)

87

In [260]:
## Use to export to Tableau
providers_final.reset_index()

,index,Provider Type,HCPCS Code,HCPCS Description,Place of Service,Average Medicare Allowed Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,Average Submitted Charge Amount,National Provider Identifier,Number of Distinct Medicare Beneficiary/Per Day Services,Number of Medicare Beneficiaries,Number of Services
0,71,Addiction Medicine,90847,"Family psychotherapy including patient, 50 min...",O,102.760000,75.680820,80.779673,120.013687,1.700878e+09,1622.0,128.0,1622.0
1,83,Addiction Medicine,95004,"Injection of allergenic extracts into skin, ac...",O,6.360746,4.800754,5.395866,8.425672,1.295891e+09,39.0,39.0,1340.0
2,33,Addiction Medicine,82575,Creatinine clearance measurement to test for k...,O,12.970000,12.710000,12.710000,25.000000,1.942314e+09,957.0,41.0,957.0
3,156,Addiction Medicine,G0397,Alcohol and/or substance (other than tobacco) ...,O,69.620000,53.877956,55.292240,139.240000,1.124073e+09,817.0,283.0,817.0
4,80,Addiction Medicine,93798,Physician services for outpatient heart rehabi...,F,13.580000,10.207731,10.823529,100.000000,1.740270e+09,595.0,31.0,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41343,55732,Vascular Surgery,62220,Creation of brain fluid drainage shunt,F,809.640625,634.763125,479.299375,4890.000000,1.316019e+09,16.0,15.0,16.0
41344,55766,Vascular Surgery,72052,"X-ray of upper spine, 6 or more views",F,17.420000,13.660000,14.350000,57.062500,1.578745e+09,16.0,16.0,16.0
41345,55884,Vascular Surgery,76856,Ultrasound of pelvis,F,32.953750,24.213750,25.233750,110.312500,1.578745e+09,16.0,15.0,16.0
41346,55952,Vascular Surgery,90732,Vaccine for pneumococcal polysaccharide for in...,O,95.240625,93.335625,93.335625,95.937500,1.205874e+09,16.0,16.0,16.0


## Create dataframe for Office ('O') "Place of Service"

In [ ]:
prov_desc_df_office = prov_desc_df.loc[prov_desc_df['Place of Service'] == 'O']

In [ ]:
##prov_desc_df_office = prov_desc_df.sort_values(by = 'Number of Services', ascending = False)

In [ ]:
prov_desc_df_office.head()

## Check there are no "F" (facility) 'Place of Service' entries.

In [ ]:
prov_desc_df_office['Place of Service'].value_counts()

## Use a dictionary for-loop to sort HCPCS descriptions by Provider Type for Office Visits.

In [ ]:
# Initialize an empty dictionary for office visits: 

provider_type_dict_ofc = {} 

# Iterate over the file
    
    # Iterate over the column in DataFrame
for key, val in prov_desc_df_office.iterrows():
    if val['Provider Type'] not in provider_type_dict_ofc.keys():
        provider_type_dict_ofc[val['Provider Type']]=[val['HCPCS Description']]
        
    elif (val['HCPCS Description'] not in provider_type_dict_ofc[val['Provider Type']]):
        provider_type_dict_ofc[val['Provider Type']].append(val['HCPCS Description'])
        
    else:
        continue


# Print the populated dictionary
print(provider_type_dict_ofc)

## Turn provider type/descriptions dictionary into a dataframe

In [ ]:
provider_type_subset_ofc = pd.DataFrame.from_dict(provider_type_dict_ofc, orient = "index")

In [ ]:
pd.options.display.max_rows = 100
provider_type_subset_ofc

## Repeat the above steps to subset the "F" ('Facility') Services 

In [ ]:
prov_desc_df_facility = prov_desc_df.loc[prov_desc_df['Place of Service'] == 'F']

In [ ]:
##prov_desc_df_facility = prov_desc_df.sort_values(by = 'Number of Services', ascending = False)

In [ ]:
prov_desc_df_facility.head()

In [ ]:
## confirm only 'Facility' entries

prov_desc_df_facility['Place of Service'].value_counts()

## Use a dictionary for-loop to sort HCPCS descriptions by Provider Type for Facilty Visits.

In [ ]:
# Initialize an empty dictionary for facility visits: 

provider_type_dict_fac = {} 

# Iterate over the file
    
    # Iterate over the column in DataFrame
for key, val in prov_desc_df_facility.iterrows():
    if val['Provider Type'] not in provider_type_dict_fac.keys():
        provider_type_dict_fac[val['Provider Type']]=[val['HCPCS Description']]
        
    elif (val['HCPCS Description'] not in provider_type_dict_fac[val['Provider Type']]):
        provider_type_dict_fac[val['Provider Type']].append(val['HCPCS Description'])
        
    else:
        continue


# Print the populated dictionary
print(provider_type_dict_fac)

In [ ]:
provider_type_subset_fac = pd.DataFrame.from_dict(provider_type_dict_fac, orient = "index")

In [ ]:
pd.options.display.max_rows = 100
provider_type_subset_fac

## There is a discrepancy between number of codes and the descriptions...

In [ ]:
prov_desc_df['HCPCS Description'].nunique()

In [ ]:
prov_desc_df['HCPCS Code'].nunique()

## Created csv files for export to Tableau to further analysis

In [ ]:
## Main dataframe
prov_desc_df_no_office.to_csv('prov_type_no_office_visit.csv')

In [ ]:
## Office services subset
provider_type_subset_ofc.to_csv('prov_type_sub_ofc.csv')

In [ ]:
## Facility services subset
provider_type_subset_fac.to_csv('prov_type_sub_fac.csv')

In [ ]:
## Office visits dataframe
prov_desc_df_office.to_csv('provider_type_office_service.csv')

## Attempt to use plot(s) to visualize some parts of the data

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
plt.bar(prov_desc_df['Provider Type'], prov_desc_df['Place of Service'])
plt.xlabel('Type')
plt.ylabel('Place of Service')
##plt.legend()
##  .set_xticklabels(years)
plt.title('Provider Types by Location', color = 'black');